In [1]:
import pandas as pd
import pydicom
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib
from pathlib import Path
import shutil

# Prepare dataset

In [2]:
train_paths = sorted(glob('/home/johnny/Documents/pyplatypus/data/dicom-images-train/*/*/*.dcm'))
test_paths = sorted(glob('/home/johnny/Documents/pyplatypus/data/dicom-images-test/*/*/*.dcm'))
data_df = pd.read_csv('/home/johnny/Documents/pyplatypus/data/train-rle.csv')
data_path = "/home/johnny/Documents/pyplatypus/data/training_data"

In [3]:
def rle2mask(rle, width, height):
    mask= np.zeros(width* height)
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        current_position += start
        mask[current_position:current_position+lengths[index]] = 255
        current_position += lengths[index]

    return mask.reshape(width, height)

In [13]:
Path(data_path).mkdir(parents=True, exist_ok=True)
idx = 0
training_dataset = dict()
width, height = 1024, 1024
not_empty_masks = list()

for path in train_paths:
    file_name = path.split('/')[-1][:-4]
    if all(data_df["ImageId"] != file_name):
        continue

    Path(f"{data_path}/image_{idx}").mkdir(parents=True, exist_ok=True)
    rle_mask = data_df[data_df["ImageId"] == file_name][" EncodedPixels"].values[0]
    if  "-1" in rle_mask:
        mask = np.zeros((width, height))
    else:
        mask = rle2mask(rle_mask, width, height)
        not_empty_masks.append(idx)
    Path(f"{data_path}/image_{idx}/images").mkdir(parents=True, exist_ok=True)
    Path(f"{data_path}/image_{idx}/masks").mkdir(parents=True, exist_ok=True)
    shutil.copyfile(path, f"{data_path}/image_{idx}/images/image_{idx}.dcm")
    matplotlib.image.imsave(f"{data_path}/image_{idx}/masks/mask_{idx}.png", mask)
    idx+=1
    if idx %100 == 0:
        print(idx)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600


# Run training

In [2]:
cfg = {
    "augmentation": {
        "Blur": {
            "always_apply": False,
            "blur_limit": 7,
            "p": 0.5
        },
        "Flip": {
            "always_apply": False,
            "p": 0.5
        },
        "RandomRotate90": {
            "always_apply": False,
            "p": 0.5
        },
        "ToFloat": {
            "always_apply": True,
            "max_value": 255,
            "p": 1.0
        }
    },
    "object_detection": None,
    "semantic_segmentation": {
        "data": {
            "colormap": [
                [
                    0,
                    0,
                    0
                ],
                [
                    255,
                    255,
                    255
                ]
            ],
            "column_sep": ";",
            "loss": "focal loss",
            "metrics": [
                "tversky coefficient",
                "iou coefficient"
            ],
            "mode": "nested_dirs",
            "optimizer": "adam",
            "shuffle": False,
            "subdirs": [
                "images",
                "masks"
            ],
            "test_path":  "/home/johnny/Documents/pyplatypus/data/training_data",
            "train_path": "/home/johnny/Documents/pyplatypus/data/training_data",
            "validation_path":  "/home/johnny/Documents/pyplatypus/data/training_data"
        },
        "models": [
            {
                "activation_layer": "relu",
                "batch_normalization": True,
                "batch_size": 32,
                "blocks": 4,
                "deep_supervision": False,
                "dropout": 0.2,
                "epochs": 200,
                "filters": 16,
                "grayscale": False,
                "h_splits": 0,
                "kernel_initializer": "he_normal",
                "linknet": False,
                "n_class": 2,
                "name": "res_u_net",
                "net_h": 256,
                "net_w": 256,
                "plus_plus": False,
                "resunet": True,
                "u_net_conv_block_width": 4,
                "use_separable_conv2d": True,
                "use_spatial_droput2d": True,
                "use_up_sampling2d": False,
                "w_splits": 0
            },
            {
                "batch_normalization": True,
                "batch_size": 8,
                "blocks": 2,
                "deep_supervision": False,
                "dropout": 0.2,
                "epochs": 200,
                "filters": 16,
                "grayscale": False,
                "h_splits": 0,
                "kernel_initializer": "he_normal",
                "linknet": False,
                "n_class": 2,
                "name": "u_net_plus_plus",
                "net_h": 256,
                "net_w": 256,
                "plus_plus": True,
                "use_separable_conv2d": True,
                "use_spatial_dropout2d": True,
                "use_up_sampling2d": True,
                "w_splits": 0
            }
        ]
    }
}
import yaml
with open("cfg.yml", "w") as outfile:
    yaml.dump(cfg, outfile, default_flow_style=False)

In [3]:
import sys
sys.path.append('/home/johnny/Documents/pyplatypus')

from pyplatypus.solvers.platypus_cv_solver import PlatypusSolver

ps = PlatypusSolver(
    config_yaml_path=Path("cfg.yml")
)
ps.train()

2022-10-19 11:23:05.880698: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-19 11:23:05.978209: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/johnny/.virtualenvs/platypus/lib/python3.9/site-packages/cv2/../../lib64:
2022-10-19 11:23:05.978242: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-19 11:23:05.993970: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when on

10675 images detected!
Set 'steps_per_epoch' to: 334
10675 images detected!
Set 'steps_per_epoch' to: 334


2022-10-19 11:23:07.340773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-19 11:23:07.340965: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/johnny/.virtualenvs/platypus/lib/python3.9/site-packages/cv2/../../lib64:
2022-10-19 11:23:07.340999: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/johnny/.virtualenvs/platypus/lib/python3.9/site-packages/cv2/../../lib64:
2022-10-19 11:23:07.341028: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libc

Epoch 1/200
  1/334 [..............................] - ETA: 1:13:19 - loss: 0.0000e+00 - categorical_crossentropy: 0.0000e+00 - iou_coefficient: 4.7684e-14

KeyboardInterrupt: 